In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics#, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor
import sys
import warnings
import itertools
import scipy

if not sys.warnoptions:
    warnings.simplefilter("ignore")

path = "/home/darragh/avito/data/"
#path = '/Users/dhanley2/Documents/avito/data/'
#path = '/home/ubuntu/avito/data/'
#data_path = 'data/'

In [ ]:
#lgb25 = pd.read_csv('../lgCV_2505.csv.gz', compression='gzip')
lgb02A = pd.read_csv(path+'../sub/lgCV_0206A.csv.gz', compression='gzip')
lgb09 = pd.read_csv(path+'../sub/lgCV_0906.csv.gz', compression='gzip')
lgb10 = pd.read_csv(path+'../sub/lgCV_1006.csv.gz', compression='gzip')
lgb11A= pd.read_csv(path+'../sub/lgCV_1106A.csv.gz', compression='gzip')
lgb11D= pd.read_csv(path+'../sub/lgCV_1106D.csv.gz', compression='gzip')
lgb14= pd.read_csv(path+'../sub/lgCV_1406.csv.gz', compression='gzip')
lgb14A= pd.read_csv(path+'../sub/lgCV_1406A.csv.gz', compression='gzip')
lgb27 = pd.read_csv(path+'../sub/lgCV_2705B.csv.gz', compression='gzip')
lgb31 = pd.read_csv(path+'../sub/lgCV_3105.csv.gz', compression='gzip')
lgb02 = pd.read_csv(path+'../sub/lgCV_0206.csv.gz', compression='gzip')
lgb17 = pd.read_csv(path+'../sub/lgCV_1706.csv.gz', compression='gzip')
lgb19 = pd.read_csv(path+'../sub/lgCV_1906.csv.gz', compression='gzip')
lgb20 = pd.read_csv(path+'../sub/lgCV_2006.csv.gz', compression='gzip')
lgb20C = pd.read_csv(path+'../sub/lgCV_2006C.csv.gz', compression='gzip')
lgb20B = pd.read_csv(path+'../sub/lgCV_2006B.csv.gz', compression='gzip')
lgb20AA = pd.read_csv(path+'../sub/lgCV_2006AA.csv.gz', compression='gzip')
lgb19A= pd.read_csv(path+'../sub/lgCV_1906A.csv.gz', compression='gzip')
rnn =   pd.read_csv(path+'../sub/rnnCV_2805.csv.gz', compression='gzip')
rnn27 = pd.read_csv(path+'../sub/rnnCV_2705A.csv.gz', compression='gzip')
rnn12 = pd.read_csv(path+'../sub/rnnCV_1206.csv.gz', compression='gzip')
rnn12 = pd.read_csv(path+'../sub/rnnCV_1206.csv.gz', compression='gzip')
mlp =   pd.read_csv(path+'../sub/mlpCV_2505.csv.gz', compression='gzip')
rmlp23A =   pd.read_csv(path+'../sub/rmlp5CV_2306A.csv.gz', compression='gzip')
rmlp23 =   pd.read_csv(path+'../sub/rmlp5CV_2306.csv.gz', compression='gzip')
rdgv19 =   pd.read_csv(path+'../sub/rdgv19CV_1606.csv.gz', compression='gzip')
rdgv20 =   pd.read_csv(path+'../sub/rdg5CV_2006B.csv.gz', compression='gzip')
rdgv23 =   pd.read_csv(path+'../sub/rdg5CV_2306.csv.gz', compression='gzip')
rdgv23A =   pd.read_csv(path+'../sub/rdg5CV_2306A.csv.gz', compression='gzip')


truth = pd.read_csv(path+'train.csv.zip', compression='zip', parse_dates = ["activation_date"])
y =     truth['deal_probability'].values
truth.drop('deal_probability', 1)
test =  pd.read_csv(path+'test.csv.zip', compression='zip', parse_dates = ["activation_date"])
test['deal_probability']=float('NAN') 
truth = pd.concat([truth,test[truth.columns]],axis=0)

In [ ]:
truth['image_on_ad'] = ((truth['image_top_1']==truth['image_top_1']).astype(np.int8))

In [ ]:
entropy = pd.read_csv(path+'../features/user_entropy_2306.gz', compression='gzip')
entropy.drop(['user_id', 'index'], 1, inplace = True)
entropy.head()

In [ ]:
lgb14['deal_probability'] =  ( lgb14['deal_probability'].values + lgb14A['deal_probability'].values)*0.5
lgb17['deal_probability'] =  ( lgb17['deal_probability'].values + lgb19['deal_probability'].values)*0.25
lgb20['deal_probability'] =  ( lgb20['deal_probability'].values) * 0.5


In [ ]:
#img_cts = truth['image_top_1'].value_counts().reset_index()
#truth['image_top_big'] = 'lo_count'
#big_top_1 = img_cts[img_cts['image_top_1']>5000]['index'].tolist()
#idx = truth['image_top_1'].isin(big_top_1)
#truth['image_top_big'][idx] = truth['image_top_1'][idx].astype(str).values

def keep_big(df, col, cutoff):
    cts = df[col].value_counts().reset_index()
    df[col+'_big'] = 'lo_count'
    big = cts[cts[col]>cutoff]['index'].tolist()
    idx = df[col].isin(big)
    df[col+'_big'][idx] = df[col][idx].astype(str).values
    return df[col+'_big'].values
truth['image_top_1_big'] = keep_big(truth, 'image_top_1', 5000)
truth['param_1_big'] = keep_big(truth, 'param_1', 5000)
truth['param_2_big'] = keep_big(truth, 'param_2', 5000)
truth['param_3_big'] = keep_big(truth, 'param_3', 5000)
truth['city_big'] = keep_big(truth, 'city', 5000)

In [ ]:
truth[[c for c in truth.columns if 'big' in c]].head()

In [ ]:
#lgb25.rename(columns={'deal_probability': 'lgb25_preds' }, inplace=True)
lgb31.rename(columns={'deal_probability': 'lgb31_preds' }, inplace=True)
lgb27.rename(columns={'deal_probability': 'lgb27_preds' }, inplace=True)
lgb02.rename(columns={'deal_probability': 'lgb02_preds' }, inplace=True)
lgb09.rename(columns={'deal_probability': 'lgb09_preds' }, inplace=True)
lgb10.rename(columns={'deal_probability': 'lgb10_preds' }, inplace=True)
lgb11D.rename(columns={'deal_probability': 'lgb11D_preds' }, inplace=True)
lgb11A.rename(columns={'deal_probability': 'lgb11A_preds' }, inplace=True)
lgb14.rename(columns={'deal_probability': 'lgb14_preds' }, inplace=True)
lgb17.rename(columns={'deal_probability': 'lgb17_preds' }, inplace=True)
lgb20.rename(columns={'deal_probability': 'lgb20_preds' }, inplace=True)
lgb20C.rename(columns={'deal_probability': 'lgb20C_preds' }, inplace=True)
lgb20B.rename(columns={'deal_probability': 'lgb20B_preds' }, inplace=True)
lgb20AA.rename(columns={'deal_probability': 'lgb20AA_preds' }, inplace=True)
lgb19A.rename(columns={'deal_probability': 'lgb19A_preds' }, inplace=True)
lgb02A.rename(columns={'deal_probability': 'lgb02A_preds' }, inplace=True)
rnn27.rename(columns={'deal_probability': 'rnn27_preds' }, inplace=True)
rnn12.rename(columns={'deal_probability': 'rnn12_preds' }, inplace=True)
rdgv19.rename(columns={'deal_probability': 'rdgv19_preds' }, inplace=True)
rdgv20.rename(columns={'deal_probability': 'rdgv20_preds' }, inplace=True)
rdgv23.rename(columns={'deal_probability': 'rdgv23_preds' }, inplace=True)
rdgv23A.rename(columns={'deal_probability': 'rdgv23A_preds' }, inplace=True)
rmlp23.rename(columns={'deal_probability': 'rmlp23_preds' }, inplace=True)
rmlp23A.rename(columns={'deal_probability': 'rmlp23A_preds' }, inplace=True)
mlp.rename(columns={'deal_probability': 'mlp_preds' }, inplace=True)
preds_df = lgb27.merge(rnn, on='item_id')\
                .merge(mlp, on='item_id')\
                .merge(rmlp23, on='item_id')\
                .merge(rmlp23A, on='item_id')\
                .merge(lgb31, on='item_id')\
                .merge(lgb02, on='item_id')\
                .merge(lgb09, on='item_id')\
                .merge(lgb10, on='item_id')\
                .merge(lgb11A, on='item_id')\
                .merge(lgb11D, on='item_id')\
                .merge(lgb14, on='item_id')\
                .merge(lgb17, on='item_id')\
                .merge(lgb20, on='item_id')\
                .merge(lgb20C, on='item_id')\
                .merge(lgb20B, on='item_id')\
                .merge(lgb20AA, on='item_id')\
                .merge(lgb19A, on='item_id')\
                .merge(lgb02A, on='item_id')\
                .merge(rnn27, on='item_id')\
                .merge(rnn12, on='item_id')\
                .merge(rdgv19, on='item_id')\
                .merge(rdgv20, on='item_id')\
                .merge(rdgv23, on='item_id')\
                .merge(rdgv23A, on='item_id')\
                .merge(truth, on='item_id',how='left')

In [ ]:
del rnn, \
                mlp, \
                rmlp23, \
                rmlp23A, \
                lgb31, \
                lgb02, \
                lgb09, \
                lgb10, \
                lgb11A, \
                lgb11D, \
                lgb14, \
                lgb17, \
                lgb20, \
                lgb20C, \
                lgb20B, \
                lgb20AA, \
                lgb19A, \
                lgb02A, \
                rnn27, \
                rnn12, \
                rdgv19, \
                rdgv20, \
                rdgv23, \
                rdgv23A

In [ ]:
pred_cols = [col for col in preds_df.columns if ('_preds' in col) \
             and ('lgb27' not in col) and ('lgb02_' not in col) and ('lgb11A_' not in col) \
             and ('rdgv19_' not in col) and ('lgb17_' not in col)]

preds_df['preds_sum'] = preds_df[pred_cols].sum(axis=1)
pred_cols

In [ ]:
preds_df['price'].fillna(-1,inplace=True)
preds_df['_max_'] = np.max(np.array([preds_df[col] for col in pred_cols]),axis=0).astype(np.float32)
preds_df['_min_'] = np.min(np.array([preds_df[col] for col in pred_cols]),axis=0).astype(np.float32)
preds_df['_avg_'] = np.mean(np.array([preds_df[col] for col in pred_cols]),axis=0).astype(np.float32)
preds_df['_med_'] = np.median(np.array([preds_df[col] for col in pred_cols]),axis=0).astype(np.float32)
preds_df['_std_'] = np.std(np.array([preds_df[col] for col in pred_cols]),axis=0).astype(np.float32)

In [ ]:
#for v in ['rnn', 'rdg', 'lgb1', 'lgb2']:
#    cols = [col for col in preds_df.columns if v in col]
#    preds_df[v+'_max_'] = np.max(np.array([preds_df[col] for col in cols]),axis=0).astype(np.float32)
#    preds_df[v+'_min_'] = np.min(np.array([preds_df[col] for col in cols]),axis=0).astype(np.float32)
#    preds_df[v+'_avg_'] = np.mean(np.array([preds_df[col] for col in cols]),axis=0).astype(np.float32)
#    preds_df[v+'_med_'] = np.median(np.array([preds_df[col] for col in cols]),axis=0).astype(np.float32)
#    preds_df[v+'_std_'] = np.std(np.array([preds_df[col] for col in cols]),axis=0).astype(np.float32)

In [ ]:
import itertools
for p1, p2 in itertools.combinations(pred_cols, 2):
    print('difference_%s__%s'%(p1,p2))
    preds_df['difference_%s__%s'%(p1,p2)] = (preds_df[p2] - preds_df[p1]).astype(np.float32)
    preds_df['sums_%s__%s'%(p1,p2)] = (preds_df[p2] + preds_df[p1]).astype(np.float32)

In [ ]:
for col in entropy.columns:
    print(col)
    preds_df[col] = (entropy[col].values).astype(np.float32)
del entropy
import gc
gc.collect()

In [ ]:
gc.collect()

In [ ]:
preds_df.head()

### Train scores

In [ ]:
pred_cols = [col for col in preds_df.columns if ('preds' in col) 
             and ('difference' not in col) 
             and ('sum' not in col)]

In [ ]:
idx = preds_df['deal_probability']==preds_df['deal_probability']
print(idx.value_counts())
for col in [c for c in preds_df.columns if ('_preds' in c) and ('difference' not in c) and ('sum' not in c)]:
    print('RMSE %s: '%(col), np.sqrt(metrics.mean_squared_error(preds_df['deal_probability'][idx].values, preds_df[col][idx].values)))

In [ ]:
foldls = [["2017-03-15", "2017-03-16", "2017-03-17"], \
       ["2017-03-18", "2017-03-19", "2017-03-20"], \
       ["2017-03-21", "2017-03-22", "2017-03-23"], \
       ["2017-03-24", "2017-03-25", "2017-03-26"], \
        ["2017-03-27", "2017-03-28", "2017-03-29", \
            "2017-03-30", "2017-03-31", "2017-04-01", \
            "2017-04-02", "2017-04-03","2017-04-07"]]
foldls = [[pd.to_datetime(d) for d in f] for f in foldls]
preds_df['fold'] = -1
for t, fold in enumerate(foldls):
    preds_df['fold'][preds_df.activation_date.isin(fold)] = t

In [ ]:
scores = []
for col in [c for c in preds_df.columns if ('_preds' in c) \
            and ('difference' not in c) \
            and ('sum' not in c)]:
    lstmp = [col]
    for i in range(5):
        idx = preds_df['fold']==i
        lstmp.append(np.sqrt(metrics.mean_squared_error(preds_df['deal_probability'][idx].values, \
                                                        preds_df[col][idx].values)))
    scores.append(lstmp)
pd.DataFrame(scores, columns = ['Model']+['Fold%s'%(i) for i in range(5)])

### Correlations in test and train

In [ ]:
# Test correlation
preds_df[~preds_df['deal_probability'].isnull()][[c for c in preds_df.columns if ('_preds' in c) \
                                                  and ('difference' not in c) and ('sum' not in c) ]].corr()

In [ ]:
# Train correlation
preds_df[preds_df['deal_probability'].isnull()][[c for c in preds_df.columns if ('_preds' in c)  \
                                                  and ('difference' not in c) and ('sum' not in c) ]].corr()

In [ ]:
cols = [c for c in preds_df.columns if '_preds' in c]
cols += [c for c in preds_df.columns if 'difference' in c]
cols += [c for c in preds_df.columns if '_entropy' in c]
cols += [c for c in preds_df.columns if 'imgnet' in c]
cols += [c for c in preds_df.columns if 'image_on_ad' in c]
cols += ['price', 'item_seq_number']
for ii in ['_max_', '_min_', '_avg_', '_std_', '_med_']:
    cols += [c for c in preds_df.columns if ii in c]
categories = ['region', 'param_1_big', 'parent_category_name', 'category_name', \
              'param_2_big', 'param_3_big', 'city_big', 'user_type', 'image_top_1_big']#,
cols += categories
cols = list(set(cols))
print(len(cols))

In [ ]:
for col in categories:
    preds_df[col] = LabelEncoder().fit_transform(preds_df[col].fillna("0"))

In [ ]:
train_df = preds_df[~preds_df['deal_probability'].isnull()]
test_df = preds_df[preds_df['deal_probability'].isnull()]

In [ ]:
n_estimators = 4000
train_X, valid_X, train_y, valid_y = train_test_split(train_df[cols], y, train_size=.8, random_state=12345)
eval_set = [(train_X,train_y),(valid_X,valid_y)]

In [ ]:
len(train_X)

In [ ]:
%%time
gc.collect()
clf = LGBMRegressor(n_estimators=n_estimators, 
                    max_depth=-1, 
                    feature_fraction= 0.4,
                    num_leaves=32, 
                    learning_rate=.01)#, device='gpu')
clf.fit(train_X, train_y, early_stopping_rounds=80, 
        eval_set=eval_set, eval_metric='rmse', verbose=100, 
        categorical_feature=categories)
# [2683]	training's rmse: 0.207184	valid_1's rmse: 0.211402
# [2551]	training's rmse: 0.207177	valid_1's rmse: 0.211261
# [2810]	training's rmse: 0.206681	valid_1's rmse: 0.211139

# [3046]	training's rmse: 0.206266	valid_1's rmse: 0.211108


In [ ]:
sorted(zip(clf.feature_importances_, train_X.columns ),key=lambda x: -x[0])

In [ ]:
n_estimators = 2800
train_X = train_df[cols]
train_y = y
eval_set = [(train_X,train_y)]
len(train_X)

In [ ]:
%%time
y_predls = []
for i in range(5):    
    del clf
    gc.collect()
    clf = LGBMRegressor(n_estimators=n_estimators, 
                    max_depth=-1, 
                    feature_fraction= 0.4,
                    num_leaves=32, 
                    seed = i, 
                    learning_rate=.01)#, device='gpu')
    clf.fit(train_X, train_y, early_stopping_rounds=80, 
        eval_set=eval_set, eval_metric='rmse', verbose=100, 
        categorical_feature=categories)
    y_predls.append(clf.predict(test_df[cols]))
    gc.collect()

In [ ]:
test_df['deal_probability'] = sum(y_predls)/len(y_predls)
test_df['deal_probability'] = np.clip(test_df['deal_probability'], .0001, .9999)
test_df[['item_id', 'deal_probability']].to_csv('../lgbbsub_2306AL2.csv.gz', compression='gzip', index=False, header=True)
test_df[['item_id', 'deal_probability']].head()